In [1]:
import pandas as pd
import os
import pandas as pd
import numpy as np
from library import *

import itertools

np.random.seed(42)


try chess classification on its own first. 

In [36]:
chess_df = pd.read_csv('CHESS_notch_bp_avg_mastoid_annotated_ica_fft.csv')
chess_df['group_id'] = chess_df['pid'] + '_' + chess_df['blockid'].astype(str)
chess_df = chess_df[(chess_df['elo-bin'] == 'lo') | (chess_df['elo-bin'] == 'hi')]
chess_df['workload'] = chess_df['elo-bin'].apply(lambda x: 0 if x == 'lo' else 1)
#chess_df = chess_df.drop(columns=['elo', 'solved', 'blockid', 'sub-block-id', 'elo-bin', 'pid', 'trial_id'])
chess_freq = produce_freq_bands(chess_df)
print(chess_freq.shape)
chess_freq

X_train, X_test, y_train, y_test = split_data(chess_freq)
X_train_scaled, X_test_scaled = scale_data(X_train, X_test)
results = classify(X_train_scaled, X_test_scaled, y_train, y_test)


(2242, 28)
(1680, 20) - (1680,) - (562, 20) - (562,)
[LightGBM] [Info] Number of positive: 1096, number of negative: 584
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000270 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5100
[LightGBM] [Info] Number of data points in the train set: 1680, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.652381 -> initscore=0.629521
[LightGBM] [Info] Start training from score 0.629521


/home/mrussell/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


In [2]:
from sklearn.model_selection import GroupShuffleSplit

def split_data(df):
    # Initial split based on pid
    X_train_list, X_test_list, y_train_list, y_test_list = [], [], [], []

    # Iterate over each unique pid
    for unique_pid in df['pid'].unique():

        # Filter data for the current pid
        pid_data = df[df['pid'] == unique_pid]
        
        X_pid = pid_data.drop(columns=[key for key in pid_data.keys() if 'ch_' not in key])
        y_pid = pid_data['workload']
        group_pid = pid_data['group_id']
        
        # Use GroupShuffleSplit to split data based on the group for the current pid
        gss = GroupShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
        train_idx, test_idx = next(gss.split(X_pid, y_pid, groups=group_pid))
        
        X_train_list.append(X_pid.iloc[train_idx])
        X_test_list.append(X_pid.iloc[test_idx])
        y_train_list.append(y_pid.iloc[train_idx])
        y_test_list.append(y_pid.iloc[test_idx])

    # Concatenate the results to get the final train and test sets
    X_train = pd.concat(X_train_list)
    X_test = pd.concat(X_test_list)
    y_train = pd.concat(y_train_list)
    y_test = pd.concat(y_test_list)

    print(f"{X_train.shape} - {y_train.shape} - {X_test.shape} - {y_test.shape}")

    return X_train, X_test, y_train, y_test

preprocessing

In [ ]:
# preprocessing steps.

# wl_df = wl_df.drop(columns=['time'])
# wl_df = wl_df[(wl_df['trialtype'] == 'nback')] #| (wl_df['trialtype'] == 'stroop')] #(wl_df['trialtype'] == 'rotation') | 

# wl_df.loc[:, 'group_id'] = wl_df['filename']
# wl_df = wl_df.drop(columns=['filename', 'correct', 'trial_id', 'trialtype'])

# # # -1 trials are not used [first three for nback and/or rest, etc.]
# wl_df = wl_df[wl_df['workload'] != '-1']

# # # 'True'/'False' is used by stroop, whereas 0,1,2,3 are used for nback and rotation
# # wl_df['workload'] = wl_df['workload'].apply(lambda x: '3.0' if x == 'True' else '0.0' if x == "False" else x)
# wl_df['workload'] = wl_df['workload'].astype(float).astype(int)

# for i, row in wl_df.iterrows():
#     if row['workload'] == 1:        
#         wl_df.loc[i, 'workload'] = 0
#     elif row['workload'] == 2 or row['workload'] == 3:
#         wl_df.loc[i, 'workload'] = 1
# wl_df
# wl_freq = produce_freq_bands(wl_df)

# print(wl_freq.shape)
# wl_freq

In [32]:
#wl_df = pd.read_csv('workload_raw_joined_preprocessed.csv', dtype={'pid':str, 'group_id':str, 'workload':int})
from sklearn.model_selection import train_test_split

# Reshape the data
eeg_channels = ['probe-0', 'probe-1', 'probe-2', 'probe-3']
data_array = []
epochs = wl_df['epoch'].unique()

for epoch in epochs:
    epoch_data = wl_df[wl_df['epoch'] == epoch][eeg_channels].values.T
    data_array.append(epoch_data)

data_array = np.array(data_array)

# Split into test and train datasets
# We'll use the group_id to ensure that the same group_id doesn't appear in both train and test datasets
# Stratify based on 'pid' to ensure even distribution
train_data, test_data, train_meta, test_meta = train_test_split(
    data_array, 
    wl_df.drop_duplicates(subset='epoch')[['workload', 'pid', 'group_id']], 
    test_size=0.2, 
    stratify=wl_df.drop_duplicates(subset='epoch')['pid'], 
    random_state=42
)

train_data.shape, test_data.shape


((2346, 4, 930), (587, 4, 930))

In [35]:
from sklearn.preprocessing import StandardScaler 

# Initialize the scaler
scaler = StandardScaler()

# Reshape the data to 2D: (number of samples, number of features)
train_data_2d = train_data.reshape(train_data.shape[0], -1)
test_data_2d = test_data.reshape(test_data.shape[0], -1)

# Fit and transform the training data
train_data_2d = scaler.fit_transform(train_data_2d)

# Transform the test data
test_data_2d = scaler.transform(test_data_2d)

# Reshape the data back to 3D for EEGNet
train_data = train_data_2d.reshape(train_data.shape)
test_data = test_data_2d.reshape(test_data.shape)

In [41]:
train_labels = train_meta['workload'].values
test_labels = test_meta['workload'].values

In [42]:
from braindecode.models import EEGNetv4

n_channels, n_samples = 4, 930  # As per your data
n_classes = 2  # 'workload' has two classes: 0 and 1

model = EEGNetv4(
    n_channels,
    n_classes,
    input_window_samples=n_samples,
    final_conv_length="auto",
)

In [47]:
import torch
from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss
import torch.nn.functional as F

# Convert data and labels to PyTorch tensors
train_data_tensor = torch.tensor(train_data, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.float32)
test_data_tensor = torch.tensor(test_data, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.float32)


# Convert binary labels to one-hot encoded format
train_labels_onehot = F.one_hot(train_labels_tensor.to(torch.int64), num_classes=2).float()
test_labels_onehot = F.one_hot(test_labels_tensor.to(torch.int64), num_classes=2).float()


# Set criterion and optimizer
criterion = BCEWithLogitsLoss()
optimizer = Adam(model.parameters(), lr=0.01)

# Training loop (simplified)
n_epochs = 50
for epoch in range(n_epochs):
    # Forward pass
    outputs = model(train_data_tensor)
    loss = criterion(outputs, train_labels_onehot)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss every few epochs (e.g., 10 epochs)
    if epoch % 5 == 0:
        print(f"Epoch [{epoch}/{n_epochs}], Loss: {loss.item():.4f}")

KeyboardInterrupt: 

In [46]:
# Forward pass on the test set
test_outputs = model(test_data_tensor)

# Convert logits to probabilities using sigmoid
test_probs = torch.sigmoid(test_outputs)

# Convert probabilities to class predictions
test_preds = (test_probs > 0.5).float()

# Compute accuracy
accuracy = (test_preds == test_labels_onehot).float().mean().item()

accuracy

0.5

In [73]:
chess_df = pd.read_csv('CHESS_notch_bp_avg_mastoid_annotated_ica_fft.csv')
chess_df['group_id'] = chess_df['pid'] + '_' + chess_df['blockid'].astype(str)
chess_df = chess_df[(chess_df['elo-bin'] == 'lo') | (chess_df['elo-bin'] == 'hi')]
chess_df['workload'] = chess_df['elo-bin'].apply(lambda x: 0 if x == 'lo' else 1)
chess_df = chess_df.drop(columns=['elo', 'solved', 'blockid', 'sub-block-id', 'elo-bin'])
chess_freq = produce_freq_bands(chess_df)

print(chess_freq.shape)
chess_freq

(2242, 83)


,ch_0_Delta-mean,ch_0_Delta-kurt,ch_0_Delta-skew,ch_0_Delta-std,ch_0_Theta-mean,ch_0_Theta-kurt,ch_0_Theta-skew,ch_0_Theta-std,ch_0_Alpha-mean,ch_0_Alpha-kurt,...,ch_3_Beta-kurt,ch_3_Beta-skew,ch_3_Beta-std,ch_3_Gamma-mean,ch_3_Gamma-kurt,ch_3_Gamma-skew,ch_3_Gamma-std,pid,group_id,workload
0,390.720847,-1.310962,0.178668,152.372900,428.479043,-0.226652,-0.017858,155.792028,253.743533,-0.931283,...,-1.491289,0.144239,38.861238,33.448872,-0.077841,0.921366,10.803862,16230396,16230396_1,0
1,398.452080,0.541613,0.271998,84.151871,340.348929,1.184020,-0.937548,83.618961,220.584748,-0.698928,...,-0.652512,0.383162,52.140729,36.279214,-0.759120,0.429387,18.971920,16230396,16230396_2,0
2,470.196750,-1.062051,0.770095,274.789355,405.067821,-0.816950,-0.427824,118.126549,239.159817,-1.090345,...,0.869889,0.838840,49.336086,41.369661,-0.542480,0.600152,23.229765,16230396,16230396_3,0
3,477.783855,-1.361172,0.464569,187.479635,440.043544,-0.348134,0.646387,193.703996,213.095992,-0.924912,...,-1.310227,-0.108096,43.726065,24.439770,-1.112022,0.026026,7.912989,16230396,16230396_3,0
4,417.306307,-0.713179,-0.619647,112.188014,335.667348,-1.215145,0.156117,140.693616,259.109848,-0.813610,...,-1.112308,0.413483,59.538573,30.329723,0.805317,0.977456,18.156995,16230396,16230396_4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3348,251.772393,-1.091685,0.364251,76.597507,202.669652,-1.225869,0.018183,60.648647,156.296846,-0.514930,...,1.814169,0.607235,15.059297,27.059622,-1.049143,0.192883,10.078352,cf88d785,cf88d785_148,1
3349,1023.303423,-0.495014,-0.718650,253.663734,277.892588,-0.008889,1.150856,174.635574,209.978572,-0.966375,...,-0.907621,0.409998,32.088369,41.923898,1.176971,1.136630,16.815894,cf88d785,cf88d785_149,1
3350,719.141696,-1.255632,-0.059061,244.069346,364.504418,-1.544349,0.418076,274.720122,309.640371,-0.175616,...,3.814900,2.083426,71.966485,42.026060,0.105331,0.788962,17.016731,cf88d785,cf88d785_149,1
3351,1668.572680,-0.850939,0.966500,1563.982014,209.108127,-1.104097,-0.055342,105.655116,166.200306,0.023359,...,-0.768537,0.546442,31.156542,80.495806,-1.087275,0.046721,27.038209,cf88d785,cf88d785_149,1


In [76]:
all_freq_data = pd.concat([chess_freq, wl_freq])  #wl_freq #
X_train, X_test, y_train, y_test = split_data(all_freq_data)
X_train_scaled, X_test_scaled = scale_data(X_train, X_test)
results = classify(X_train_scaled, X_test_scaled, y_train, y_test)

print(results)

(2137, 80) - (2137,) - (796, 80) - (796,)
[LightGBM] [Info] Number of positive: 1162, number of negative: 975
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000762 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 20400
[LightGBM] [Info] Number of data points in the train set: 2137, number of used features: 80
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.543753 -> initscore=0.175460
[LightGBM] [Info] Start training from score 0.175460


/home/mrussell/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


svc         0.652010
log_reg     0.594902
rf          0.827889
knn         0.660770
ann         0.752371
gbm         0.738414
lightgbm    0.817837
xgboost     0.821604
dtype: float64


| Data  |      Classifier      | macro f1 |
|:--------:|:----------------------:|:-----------:|
| Nback	       | lightgbm  |    0.82 |
| Nback + Rotation | rf | 0.76 |
| Chess + Nback |   rf      |    0.75 |
| Nback + Stroop | rf | 0.72 |
| Nback + Stroop + Rotation	       |   rf	     | 0.67 |
| All          |   rf      |    0.67 |
| Chess + Rotation | rf | 0.64 |
| Rotation	   | xgboost	  |    0.63 |
| Nback (4-class) | lightgbm | 0.62 |
| Stroop	   |   rf	     | 0.62 |
| Chess        |   rf      |    0.62 |
| Chess + Rotation + Stroop | rf | 0.62 |
| Chess + Stroop | rf | 0.62 |
| Stroop + Rotation | rf | 0.59 |
| Chess + Nback + Stroop | rf | 0.57
| Chess + Nback + Rotation | rf | 0.55 |


with added kurtosis, skew, and std
| Data  |      Classifier      | macro f1 |
|:--------:|:----------------------:|:-----------:|
| Nback	       | rf |    0.83 |

In [77]:

def produce_freq_bands(df):

    # Define the frequency bands
    freq_bands = {
        "Delta": (0.5, 4),
        "Theta": (4, 8),
        "Alpha": (8, 14),
        "Beta":  (14, 30),
        "Gamma": (30, 45)
    }

    # Extract only the columns with frequency data
    freq_data = df.filter(like="freq")

    binned_df = pd.DataFrame()

    for ch in range(4):  # Assuming you have 4 channels: ch_0, ch_1, ch_2, ch_3
        for band, (f_min, f_max) in freq_bands.items():
            # Filter columns for the current channel and band
            cols = [col for col in freq_data.columns 
                    if f"ch_{ch}_freq_" in col 
                    and f_min <= float(col.split("_")[3][:-2]) <= f_max]
            
            # Aggregate the columns and add to the binned dataframe
            if cols:
                binned_df[f"ch_{ch}_{band}-mean"] = freq_data[cols].mean(axis=1)
                binned_df[f"ch_{ch}_{band}-kurt"] = freq_data[cols].kurt(axis=1)
                binned_df[f"ch_{ch}_{band}-skew"] = freq_data[cols].skew(axis=1)
                binned_df[f"ch_{ch}_{band}-std"] = freq_data[cols].std(axis=1)
            else:
                binned_df[f"ch_{ch}_{band}"] = np.nan

    # Add back non-frequency columns to the binned dataframe
    for col in [key for key in df.keys() if key not in freq_data]:
        binned_df[col] = df[col]

    df = binned_df
    return df 

within subjects test; make sure to separate data into EITHER test/train based on which file produced the data. technically could get more specific with nback task, but this is generalizable.  

In [31]:
from sklearn.preprocessing import StandardScaler

def scale_data(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled


In [21]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb
import xgboost as xgb

def classify(X_train_scaled, X_test_scaled, y_train, y_test):
    models = {  
                # 'svc': SVC(), 
                # 'log_reg': LogisticRegression(max_iter=10000), 
                'rf': RandomForestClassifier(), 
                'knn': KNeighborsClassifier(n_neighbors=5),  # Start with 5 neighbors, 
                'ann': MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=10000),  # Simple 2-layer feedforward network,
                'gbm': GradientBoostingClassifier(), 
                'lightgbm': lgb.LGBMClassifier(), 
                'xgboost': xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
            }

    results = {model: None for model in models.keys()}

    for model_name, model in models.items():
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        f1 = f1_score(y_test, y_pred, average='macro')    
        results[model_name] = f1

    results = pd.Series(results)
    return results
